# Spam Detector

In [68]:
import os
import warnings

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore', message="KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.")

os.environ['OMP_NUM_THREADS'] = '1'

## Retrieve the Data

The data is located at [https://static.bc-edx.com/ai/ail-v-1-0/m13/challenge/spam-data.csv](https://static.bc-edx.com/ai/ail-v-1-0/m13/challenge/spam-data.csv)

Dataset Source: [UCI Machine Learning Library](https://archive.ics.uci.edu/dataset/94/spambase)

Import the data using Pandas. Display the resulting DataFrame to confirm the import was successful.

In [69]:
# Import the data
data = pd.read_csv("https://static.bc-edx.com/ai/ail-v-1-0/m13/challenge/spam-data.csv")
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


## Predict Model Performance

I expect Random Forest to perform better. Logistic Regression is a simpler linear model suitable for linear relationships, while Random Forest is a more complex ensemble model that can handle non-linear relationships and provide robust predictions in various scenarios. Random Forest is primarily for classification tasks where there are non-linear relationships between features and targets. For this reason it is often used for fraud detection, or in this case Spam detection, and so I would expect it to perform better.

*Replace the text in this markdown cell with your predictions, and be sure to provide justification for your guess.*

## Split the Data into Training and Testing Sets

In [70]:
# Create the labels set `y` and features DataFrame `X`
y = data['spam']
X = data.drop(columns=['spam'])


In [71]:
# Check the balance of the labels variable (`y`) by using the `value_counts` function.
print(y.value_counts())


spam
0    2788
1    1813
Name: count, dtype: int64


In [72]:
# Split the data into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head()




,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
1370,0.09,0.0,0.09,0.0,0.39,0.09,0.09,0.00,0.19,0.29,...,0.0,0.000,0.139,0.0,0.310,0.155,0.0,6.813,494,1458
3038,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.124,0.124,0.0,0.000,0.000,0.0,1.800,8,45
2361,0.00,0.0,2.43,0.0,0.00,0.00,0.00,0.00,0.27,0.00,...,0.0,0.000,0.344,0.0,0.000,0.000,0.0,2.319,12,167
156,0.00,0.0,0.00,0.0,1.31,0.00,1.31,1.31,1.31,1.31,...,0.0,0.000,0.000,0.0,0.117,0.117,0.0,48.500,186,291
2526,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.000,0.000,0.0,0.000,0.000,0.0,2.307,8,30


## Scale the Features

Use the `StandardScaler` to scale the features data. Remember that only `X_train` and `X_test` DataFrames should be scaled.

In [73]:
# Create the StandardScaler instance
scaler = StandardScaler()


In [74]:
# Fit the Standard Scaler with the training data
scaler.fit(X_train)


StandardScaler()

In [75]:
# Scale the training data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5]

array([[-3.84417967e-02, -1.62823436e-01, -3.73283587e-01,
        -4.43387016e-02,  1.25329068e-01, -1.75557108e-02,
        -4.94618697e-02, -2.58223628e-01,  3.74143676e-01,
         7.84018383e-02,  1.64169725e+00, -7.44333103e-02,
        -3.27398098e-01,  1.51199468e+00, -1.90055678e-01,
         7.05973261e-01,  1.15944336e-01, -3.45099898e-01,
         6.08468580e-03,  7.83188746e+00,  7.10708263e-01,
        -1.16300745e-01,  8.23756701e-01,  2.01038747e-01,
        -3.26710260e-01, -3.00878899e-01, -2.30922222e-01,
        -2.31071959e-01, -1.72682655e-01, -2.24879107e-01,
        -1.74395864e-01, -1.43223289e-01, -1.73866411e-01,
        -1.45765474e-01, -1.93393004e-01, -2.42335411e-01,
        -3.30036623e-01, -5.64810168e-02, -1.78189932e-01,
        -1.86445138e-01, -1.21635838e-01, -1.74633824e-01,
        -2.03750279e-01, -1.27828730e-01, -2.99465798e-01,
        -2.11088191e-01, -7.04194454e-02, -1.17147007e-01,
        -1.57562225e-01, -8.70397336e-03, -1.73306666e-0

## Create and Fit a Logistic Regression Model

Create a Logistic Regression model, fit it to the training data, make predictions with the testing data, and print the model's accuracy score. You may choose any starting settings you like. 

In [76]:
# Train a Logistic Regression model and print the model score
lr_model = LogisticRegression()

#Fit the model
lr_model.fit(X_train_scaled, y_train)


LogisticRegression()

In [77]:
# Make and save testing predictions with the saved logistic regression model using the test data
y_pred = lr_model.predict(X_test_scaled)

# Review the predictions
print(y_pred)



[0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1
 1 0 1 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1
 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1
 1 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 0 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1
 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1
 0 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 1 1
 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 1
 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0
 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 1 1 0
 1 1 1 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1
 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1
 0 1 1 1 0 0 0 0 0 1 1 1 

In [78]:
# Calculate the accuracy score by evaluating `y_test` vs. `testing_predictions`.
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9196525515743756


## Create and Fit a Random Forest Classifier Model

Create a Random Forest Classifier model, fit it to the training data, make predictions with the testing data, and print the model's accuracy score. You may choose any starting settings you like. 

In [82]:
# Train a Random Forest Classifier model and print the model score
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)


In [80]:
# Make and save testing predictions with the saved logistic regression model using the test data
rf_model.fit(X_train_scaled, y_train)

y_pred = rf_model.predict(X_test_scaled)


# Review the predictions
print(y_pred)


[0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 0 1
 1 0 1 1 0 0 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1
 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 1 0 0 1 1 0 0
 1 1 0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0
 0 1 0 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1
 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1
 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 0 1 1
 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 1
 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 0
 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 1 1 0
 1 1 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0
 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 1
 0 1 1 1 0 0 0 0 0 1 1 1 

In [81]:
# Calculate the accuracy score by evaluating `y_test` vs. `testing_predictions`.
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9554831704668838


## Evaluate the Models

As expected the Random Forest model performed better with an accuracy of 95.55% as opposed to the Logistic regression model accuracy of 91.97%. This outcome shows the strength of using multiple decision trees to capture a broader range of data patterns in complex datasets where interactions between features play a significant role in prediction.

*Replace the text in this markdown cell with your answers to these questions.*